# Finding appropriate Threshold using Frequency Domain Bootstrap Method 

In [ ]:
from scipy.fft import fft, ifft
from scipy.stats import norm
import numpy as np

def compute_power_spectrum(time_series):
    """Compute the power spectrum of the input time series using Hann window."""

    # Compute FFT
    fft_values = np.fft.fft(time_series)
    # Compute Power Spectrum
    power_spectrum = np.abs(fft_values) ** 2 / len(time_series)

    return power_spectrum

# Step 1: Compute power spectrum for each time series and average them
# time_series is an array of shape (19, x)
power_spectra = np.array([compute_power_spectrum(eeg_data[i, :]) for i in range(eeg_data.shape[0])])

avg_spectrum = np.mean(power_spectra, axis=0)

def compute_whitened_residuals(time_series, avg_spectrum):
    """Compute the whitened residuals of the time series."""
    spectrum = fft(time_series)
    residuals = ifft(spectrum / np.sqrt(avg_spectrum))  # No sqrt, just dividing by avg_spectrum
    return residuals

# Step 2: Compute the whitened residuals for each time series
whitened_residuals = np.array([compute_whitened_residuals(eeg_data[i, :], avg_spectrum) for i in range(eeg_data.shape[0])])

def resample_with_replacement(data):
    """Resample data with replacement."""
    indices = np.random.randint(0, data.shape, data.shape)
    return data[indices]

n_nodes = 19
n_data_points = eeg_data.shape[1]
n_surrogates = 100
surrogate_data = np.zeros((n_surrogates, n_nodes, n_data_points), dtype=complex)
    
for i in range(n_surrogates):
    for node in range(n_nodes):
        # Resample residuals and compute surrogate data
        resampled_residuals = resample_with_replacement(whitened_residuals[node])
        surrogate_spectrum = fft(resampled_residuals) * np.sqrt(avg_spectrum)  # Use avg_spectrum directly
        surrogate_data[i, node] = ifft(surrogate_spectrum)

surrogate_datar = abs(surrogate_data)

n_surrogate_epochs = n_seizure_epochs + n_background_epochs+3
plv_matrix_surrogate = np.zeros((19, 19, n_surrogate_epochs, high_freq-low_freq, n_surrogates))

for surrog_num in range(n_surrogates):
    for freq in range(low_freq, high_freq):
        freq_n = freq-low_freq

        # Band-pass filter the data in the specified frequency band
        eeg_data_filtered = bandpass_filter(surrogate_datar[surrog_num, :, :], sampfreq, low_freq, low_freq+1)
            
        # Compute the analytic signal (Hilbert transform) to get the phase
        analytic_signal = hilbert(eeg_data_filtered, axis=1)
        phase_data = np.angle(analytic_signal)

        phase_Seg = phase_data.copy()

        surrogate_phase_epochs, _ = epoch_data(phase_Seg, sampfreq, epoch_length)

        epoch_n = 0
        for epoch in surrogate_phase_epochs:
            plv_matrix_surrogate[:, :, epoch_n, freq_n, surrog_num] = calculate_plv(epoch)
            epoch_n = epoch_n + 1

plv_surrogate_all = np.mean(plv_matrix_surrogate, axis = 3)

surrogate_plv = np.mean(plv_surrogate_all, axis = 2)

p_values_back = np.zeros((n_nodes, n_nodes, n_background_epochs))

for epoch in range(n_background_epochs):
    for i in range(n_nodes):
        for j in range(i+1, n_nodes):
            surrogate_distribution = surrogate_plv[i, j, :]
            original_value = plv_back_all[i, j, epoch]
            p_values_back[i, j, epoch] = np.mean(surrogate_distribution >= original_value)
            p_values_back[j, i, epoch] = p_values_back[i, j, epoch]  # Since it's symmetric

# Many times repetition
p_values_seiz = np.zeros((n_nodes, n_nodes, n_seizure_epochs))

for epoch in range(n_seizure_epochs):
    for i in range(n_nodes):
        for j in range(i+1, n_nodes):
            surrogate_distribution = surrogate_plv[i, j, :]
            original_value = plv_seiz_all[i, j, epoch]
            p_values_seiz[i, j, epoch] = np.mean(surrogate_distribution >= original_value)
            p_values_seiz[j, i, epoch] = p_values_seiz[i, j, epoch]  # Since it's symmetric

plv_seiz_bi= (p_values_seiz<0.05).astype(int)
plv_back_bi= (p_values_back<0.05).astype(int)
